In [1]:
import collections
import copy
import datetime
import itertools
import sys
import types

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Season = cibblbibbl.season.Season
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.init()

In [4]:
S1 = max(G.seasons)
print(f'Y{S1.year.nr} {S1.name} -- {S1}')

Y7 Winter -- Season('cibbl', 7, 1)


In [5]:
# run if next season was kicked off
S1 = S1.prev
print(f'Y{S1.year.nr} {S1.name} -- {S1}')

Y6 Autumn -- Season('cibbl', 6, 4)


In [6]:
print("Target Tournaments:")
for T in sorted(S1.tournaments.values()):
    print(T)

Target Tournaments:
[51681] CIBBL - Y6, Autumn - Iron Division
[51682] CIBBL - Y6, Autumn - Tin Division
[51683] CIBBL - Y6, Autumn - Copper Division
[51684] CIBBL - Y6, Autumn - Bronze Division
[51685] CIBBL - Y6, Autumn - Silver Division
[51686] CIBBL - Y6, Autumn - Gold Division
[51691] CIBBL - Y6, Autumn - Marble Division
[51692] CIBBL - Y6, Autumn - Granite Division
[51693] CIBBL - Y6, Autumn - Clay Division
[51694] CIBBL - Y6, Autumn - Wood Division
[51695] CIBBL - Y6, Autumn - Cloth Division
[51696] CIBBL - Y6, Autumn - Paper Division
[51697] CIBBL - Y6, Autumn - Leather Division
[51698] CIBBL - Y6, Autumn - Mud Division
[51699] CIBBL - Y6, Autumn - Dust Division
[52455] CIBBL - Y6, Autumn - Youngbloods XXV
[52456] CIBBL - Y6, Autumn - Youngbloods XXVI


In [7]:
# set manually
S0 = Season('cibbl', 5, 4)
print(f'Y{S0.year.nr} {S0.name} -- {S0}')

Y5 Autumn -- Season('cibbl', 5, 4)


In [8]:
print("Target Tournaments:")
for T in sorted(S0.tournaments.values()):
    print(T)

Target Tournaments:
[47496] CIBBL - Y5, Autumn - Cloth Division
[47497] CIBBL - Y5, Autumn - Wood Division
[47498] CIBBL - Y5, Autumn - Clay Division
[47499] CIBBL - Y5, Autumn - Granite Division
[47500] CIBBL - Y5, Autumn - Marble Division
[47501] CIBBL - Y5, Autumn - Iron Division
[47502] CIBBL - Y5, Autumn - Tin Division
[47503] CIBBL - Y5, Autumn - Copper Division
[47504] CIBBL - Y5, Autumn - Bronze Division
[47505] CIBBL - Y5, Autumn - Silver Division
[47506] CIBBL - Y5, Autumn - Gold Division
[47704] CIBBL - Y5, Autumn - Youngbloods XVIII


In [9]:
CHAIN_STRINGS = {
    "Iron Division", 
    "Tin Division", 
    "Copper Division", 
    "Bronze Division", 
    "Silver Division", 
    "Gold Division", 
    "Marble Division", 
    "Granite Division", 
    "Clay Division", 
    "Wood Division", 
    "Cloth Division", 
    "Paper Division", 
    "Leather Division", 
    "Mud Division", 
    "Dust Division", 
    "Youngbloods", 
}

In [ ]:
pairs = {k: [None, []] for k in CHAIN_STRINGS}
for k in CHAIN_STRINGS:
    for T0 in sorted(S0.tournaments.values()):
        if k in T0.name:
            pairs[k][0] = T0
            break
    for T1 in sorted(S1.tournaments.values()):
        if k in T1.name:
            pairs[k][1].append(T1)
for k, [T0, LT1] in pairs.items():
    if not T0 or not LT1:
        continue
    print(f'{k}: {T0.Id} -> {", ".join(str(T.Id) for T in LT1)}')

Iron Division: 47501 -> 51681
Bronze Division: 47504 -> 51684
Tin Division: 47502 -> 51682
Gold Division: 47506 -> 51686
Granite Division: 47499 -> 51692
Clay Division: 47498 -> 51693
Wood Division: 47497 -> 51694
Marble Division: 47500 -> 51691
Cloth Division: 47496 -> 51695
Copper Division: 47503 -> 51683
Silver Division: 47505 -> 51685
Youngbloods: 47704 -> 52455, 52456


In [11]:
# comment to skip from being copied
KEYS_TO_BE_COPIED = {
    "bestplayersname",
    "code",
    "image744",
    "image873",
    "image960",
    "ppos",
}
KEYS_TO_BE_SET = {
    "Youngbloods": {"suffix"},
}

In [12]:
TYPECASTS = {
    cibblbibbl.jsonfile.JSONFileArray: list,
}

In [19]:
for k0, t in pairs.items():
    if not all(t):
        continue
    T0, LT1 = t
    for T1 in LT1:
        print(f'{T0} -> {T1}')
        for k1 in KEYS_TO_BE_COPIED:
            v = T0.config[k1]
            if type(v) in TYPECASTS:
                v = TYPECASTS[type(v)](v)
            print(f'{k1}={v!r}')
            T1.config[k1] = v
        for k2, sk3 in KEYS_TO_BE_SET.items():
            if k2 != k0:
                continue
            for k3 in sk3:
                if k3 == "suffix":
                    v = T1.name.split("-")[-1].split()[-1]
                    print(f'{k3}={v!r}')
                    T1.config[k3] = v

[47501] CIBBL - Y5, Autumn - Iron Division -> [51681] CIBBL - Y6, Autumn - Iron Division
code='D06'
ppos=[50, 40, 20, 10]
image960='/i/530115'
bestplayersname='Forged Greatness'
image744='/i/530115'
image873='/i/530115'
[47504] CIBBL - Y5, Autumn - Bronze Division -> [51684] CIBBL - Y6, Autumn - Bronze Division
code='D03'
ppos=[70, 50, 30, 10]
image960='/i/519279'
bestplayersname='Bronze Excellence'
image744='/i/519279'
image873='/i/519279'
[47502] CIBBL - Y5, Autumn - Tin Division -> [51682] CIBBL - Y6, Autumn - Tin Division
code='D05'
ppos=[60, 40, 20, 10]
image960='/i/530114'
bestplayersname='Terrific Might of Tin'
image744='/i/530114'
image873='/i/530114'
[47506] CIBBL - Y5, Autumn - Gold Division -> [51686] CIBBL - Y6, Autumn - Gold Division
code='D01'
ppos=[100, 70, 50, 30]
image960='/i/519221'
bestplayersname='Gold Excellence'
image744='/i/519221'
image873='/i/519221'
[47499] CIBBL - Y5, Autumn - Granite Division -> [51692] CIBBL - Y6, Autumn - Granite Division
code='D08'
ppos=[